### Genetic Algorithm for solving (11) in the direct method

In [ ]:
import random
import numpy as np
from deap import base, creator, tools, algorithms
from scipy.stats import norm
from scipy.linalg import solve_discrete_are, inv
import matplotlib.pyplot as plt

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)

# Problem parameters
A = np.array([[1, 0.5], [0, 1]])  # dynamics matrix 
B = np.array([[0], [0.5]])      # input matrix
N = 100  # Number of time steps
M = 100  # Number of disturbance sequences
theta = 0.05  # Quantile parameter
threshold = 1.12  # User-defined threshold for ||K * e[t]||_2

# Parameters for the GA
u_min = -10  # Lower bound of each element of state-feedback gain K
u_max = 10   # Upper bound of each element of state-feedback gain K
population_size = 150  # Increased population size
generations = 50      # Increased number of generations
cxpb = 0.8             # Crossover probability
mutpb = 0.1            # Mutation probability

# Generate disturbance sequences w^j(t)
def disturbance_samples(mean, var, shape, theta, M, N):
    dist1 = norm(loc=mean, scale=np.sqrt(var))  # Normal distribution
    samples_normal = [dist1.rvs(size=(M, N)) for _ in range(1)]

    gamma_samples = [np.random.gamma(shape=shape, scale=theta, size=(M, N)) * np.random.choice([-1, 1], size=(M, N)) for _ in range(1)]

    # Stack all samples along the last axis
    return np.stack(samples_normal + gamma_samples, axis=-1)  # Shape: (M, N, 12)

# Parameters for calling disturbance_samples
mean = -0.01
var = 0.005
shape = 5.5
theta_g = 0.005

# Generate disturbances
w = disturbance_samples(mean, var, shape, theta_g, M, N)  # Shape: (M, N, 12)

# Discrete-time LQR INITIALIZATION
Q = np.eye(2)  # State LQR cost matrix
R = np.eye(1)   # Control LQR cost matrix

# Solve the discrete-time algebraic Riccati equation (DARE)
P = solve_discrete_are(A, B, Q, R)

# Compute the LQR gain for initialization of the GA
K_LQR = -np.dot(inv(R + B.T @ P @ B), B.T @ P @ A)

# State dynamics
def state_dynamics(K, e, w_t):
    A_BK = A + B @ K
    e_next = A_BK @ e + w_t
    return e_next

# Compute R^j for a given K
def compute_R_j(K, j):
    e = np.zeros((N + 1, A.shape[0]))  # Shape: (N+1, 12)
    A_BK = A + B @ K
    w_j = w[j]  # Shape: (N, 12)
    for t in range(N):
        e[t + 1] = A_BK @ e[t] + w_j[t]
    return np.max(np.linalg.norm(e[1:], axis=1))

# Compute Ru^j for a given K
def compute_Ru_j(K, j):
    e = np.zeros((N + 1, A.shape[0]))   # Shape: (N+1, 12)
    Ke = np.zeros((N, K.shape[0]))      # Shape: (N, 6)
    A_BK = A + B @ K
    w_j = w[j]  # Shape: (N, 12)
    for t in range(N):
        e[t + 1] = A_BK @ e[t] + w_j[t]
        Ke[t] = K @ e[t]
    return np.max(np.linalg.norm(Ke[1:], axis=1))

# Objective function to be minimized
def objective_function(K_flat, theta, M, threshold):
    K = np.array(K_flat).reshape(B.shape[1], A.shape[0])  # Shape: (6, 12)
    R_values = [compute_R_j(K, j) for j in range(M)]
    Ru_values = [compute_Ru_j(K, j) for j in range(M)]
    quantile_value = np.quantile(R_values, 1 - theta)
    quantile_value_for_u = np.quantile(Ru_values, 1 - theta)

    # Penalize if the constraint is violated
    penalty = 0
    if quantile_value_for_u > threshold:
        penalty += 1000 * (quantile_value_for_u - threshold)  # Adjusted penalty scaling

    return quantile_value + np.sqrt(10) * quantile_value_for_u + penalty, # gamma = sqrt(10)

# Custom population initialization
def initialize_population(toolbox, population_size, K_initial, init_ratio=0.1):
    num_initialized = int(population_size * init_ratio)
    population = []

    K_initial_flat = K_initial.flatten()

    # Create individuals initialized with K_initial
    for _ in range(num_initialized):
        individual = creator.Individual(K_initial_flat.tolist())
        population.append(individual)

    # Create the rest of the population randomly
    num_random = population_size - num_initialized
    for _ in range(num_random):
        individual = toolbox.individual()
        population.append(individual)

    return population

# Genetic Algorithm Setup
def genetic_algorithm_solver(theta, M, u_min, u_max, threshold, K_initial, population_size=100, generations=50, cxpb=0.7, mutpb=0.2):
    # Ensure the classes are not created multiple times
    if not hasattr(creator, 'FitnessMin'):
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    if not hasattr(creator, 'Individual'):
        creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("attr_float", random.uniform, u_min, u_max)
    num_genes = B.shape[1] * A.shape[0]  # Total number of genes in the individual
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, num_genes)
    toolbox.register("population", initialize_population, toolbox=toolbox, population_size=population_size, K_initial=K_initial, init_ratio=0.1)

    toolbox.register("evaluate", objective_function, theta=theta, M=M, threshold=threshold)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.5, indpb=0.1)
    toolbox.register("select", tools.selTournament, tournsize=3)

    population = toolbox.population()

    # Stats for tracking
    stats = tools.Statistics(lambda ind: ind.fitness.values[0])
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # Running the algorithm
    population, logbook = algorithms.eaSimple(population, toolbox, cxpb=cxpb, mutpb=mutpb,
                                              ngen=generations, stats=stats, verbose=True)

    return population, logbook

# Running the solver
population, logbook = genetic_algorithm_solver(theta, M, u_min, u_max, threshold, K_LQR, population_size, generations, cxpb, mutpb)

# Best solution
best_individual = tools.selBest(population, 1)[0]
best_K = np.array(best_individual).reshape(B.shape[1], A.shape[0])
print('Best individual (K):', best_K)
print('Best fitness:', best_individual.fitness.values[0])


# Evaluate Best GA Controller
ga_fitness = objective_function(best_K.flatten(), theta, M, threshold)[0]
print('Best GA Controller Fitness:', ga_fitness)